# __FINAL PROJECT PHASE II__

# __RESEARCH QUESTION:__
Can we predict presence and rank of of the song on Spotify charts of different artists based on factors like danceability, valence, and energy?

### Importing:

In [1]:
# imports and settings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


## Data Description:
Source: https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023/data 

We are using a dataset that contains detailed information on the most famous songs of 2023, specifically focusing on their performance on major music platforms like Spotify, Apple Music, Deezer, and Shazam. It includes key metadata such as the song's name, artist(s), and release date, along with streaming data, playlist presence, and chart rankings across platforms. Additionally, the dataset offers a range of audio features, including tempo (BPM), key, mode (major/minor), and percentages indicating danceability, energy, acousticness, and other sonic characteristics, providing a comprehensive view of each track's musical and commercial success.



In [4]:
music_df = pd.read_csv('spotify-2023.csv', encoding='ISO-8859-1')
print(music_df.shape)

(953, 24)
